In [2]:
%%capture
# scaricamento dati della f1.
# preso il mugello siccome hanno giraato nel 2020
# anche loro ed è il tempio della motogp

import fastf1
import polars as pl
from pathlib import Path


# crea una cartella 'cache' per velocizzare le esecuzioni future)
fastf1.Cache.enable_cache("cache")

# Q are the qualyfin
session = fastf1.get_session(2020, "Tuscan", "Q")
session.load()


driver = "LEC"
lap = session.laps.pick_driver(driver).pick_fastest()
telemetry = lap.get_telemetry()

# inutile importaare colonna "DRS"
df = pl.DataFrame(
    telemetry[["Time", "Speed", "RPM", "nGear", "Throttle", "Brake", "X", "Y", "Z"]]
)

# scale dei dati con rpm maggiori per "simulare" di più le moto
df = df.with_columns(
    [
        (pl.col("Time").dt.total_milliseconds() / 1000).alias("Time_Sec"),
        (pl.col("RPM") * 1.1).alias("RPM"),
    ]
)

parquet_name = "telemetry_mugello_f1_2020.parquet"

output_dir = Path.cwd() / "data"
output_dir.mkdir(parents=True, exist_ok=True)

df.write_parquet(output_dir / "mugello_telemetry_2020_f1_LEC.parquet")


In [3]:
df.head(10)  # type: ignore

Time,Speed,RPM,nGear,Throttle,Brake,X,Y,Z,Time_Sec
duration[ns],f64,f64,i64,f64,bool,f64,f64,f64,f64
0ns,302.704166,11918.206671,8,100.0,false,-975.26265,-376.235906,3028.122794,0.0
71ms,303.0,11913.0,8,100.0,false,-949.153334,-322.811421,3028.7839,0.071
207ms,303.566667,11920.48,8,100.0,false,-898.0,-220.0,3030.0,0.207
311ms,304.0,11926.2,8,100.0,false,-857.872403,-140.957297,3030.885684,0.311
427ms,304.0,11976.176667,8,100.0,false,-813.0,-53.0,3032.0,0.427
551ms,304.0,12029.6,8,100.0,false,-766.075722,39.97442,3033.559652,0.551
647ms,304.4,12025.64,8,100.0,false,-730.0,112.0,3035.0,0.647
791ms,305.0,12019.7,8,100.0,false,-675.667539,221.127492,3037.504216,0.791
867ms,305.633333,12041.296667,8,100.0,false,-647.0,279.0,3039.0,0.867


In [4]:
# siccome ci sono 8 marce in f1 e la moto ne ha 6, procedo a portare a 6 qualsiasi cosa maggiore di 6
df = df.with_columns(
    [
        pl.col("nGear").clip(1,6).alias("nGear")
    ]
)
df.head(10)

Time,Speed,RPM,nGear,Throttle,Brake,X,Y,Z,Time_Sec
duration[ns],f64,f64,i64,f64,bool,f64,f64,f64,f64
0ns,302.704166,11918.206671,6,100.0,false,-975.26265,-376.235906,3028.122794,0.0
71ms,303.0,11913.0,6,100.0,false,-949.153334,-322.811421,3028.7839,0.071
207ms,303.566667,11920.48,6,100.0,false,-898.0,-220.0,3030.0,0.207
311ms,304.0,11926.2,6,100.0,false,-857.872403,-140.957297,3030.885684,0.311
427ms,304.0,11976.176667,6,100.0,false,-813.0,-53.0,3032.0,0.427
551ms,304.0,12029.6,6,100.0,false,-766.075722,39.97442,3033.559652,0.551
647ms,304.4,12025.64,6,100.0,false,-730.0,112.0,3035.0,0.647
791ms,305.0,12019.7,6,100.0,false,-675.667539,221.127492,3037.504216,0.791
867ms,305.633333,12041.296667,6,100.0,false,-647.0,279.0,3039.0,0.867


In [5]:
df.describe()

statistic,Time,Speed,RPM,nGear,Throttle,Brake,X,Y,Z,Time_Sec
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""","""663""",663.0,663.0,663.0,663.0,663.0,663.0,663.0,663.0,663.0
"""null_count""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""","""0:00:38.122484""",247.594596,11994.683904,5.644042,83.290719,0.107089,-1458.002975,-1818.180611,3120.922904,38.122484
"""std""",null,46.44246,701.854992,0.629096,31.415795,null,2647.329811,3614.449624,112.211405,22.068684
"""min""","""0:00:00""",141.0,9991.3,4.0,0.0,0.0,-5622.540581,-7746.080753,2898.901935,0.0
"""25%""","""0:00:19.111000""",211.85,11695.2,5.0,82.0,null,-3470.623969,-4830.0,3037.0,19.111
"""50%""","""0:00:38.167000""",263.0,12146.2,6.0,99.0,null,-2074.0,-2184.0,3121.602562,38.167
"""75%""","""0:00:57.247000""",283.716668,12549.491429,6.0,100.0,null,1190.0,774.0,3191.10649,57.247
"""max""","""0:01:16.270000""",312.0,13200.0,6.0,100.0,1.0,3330.0,5418.0,3330.085494,76.27


In [6]:
import plotly.express as px

fig = px.scatter(
    df,
    x="X",
    y="Y",
    color="Speed", # colora in base alla velocita
    title="Mugello",
    color_continuous_scale="Turbo",  # rosso: veloce, blu:lento)
    width=800,
    height=800,
    hover_data=["RPM", "nGear", "Time_Sec"] # sono dati extraa qundo si passa col mouse sopra
)

fig.update_xaxes(visible=False) # toglie assi dal plot
fig.update_yaxes(visible=False)

fig.update_layout(
    template="plotly_dark",           
    plot_bgcolor="rgba(0,0,0,0)",     
    paper_bgcolor="rgba(0,0,0,0)",
    yaxis=dict(
        scaleanchor="x", 
        scaleratio=1  # scaleratio=1 così forza 1 metro su X a essere uguale a 1 metro su Y.
    )
)

fig.show()

scorrendo il mouse sul grafico, si può notare come ci siano praticamente solo 5 e 6 marcia, cosa un po infattibile per una moto. Vado a verificare nel df

In [7]:
df["nGear"].value_counts()

nGear,count
i64,u32
5,126
4,55
6,482


come pensavo, forse meglio ridimensionare in maniera più intelligente, alemno per includere una terza marcia. Faccio un offset di 2 $$8-2 = 6:  n marce moto$$

In [8]:
df = pl.read_parquet("data/mugello_telemetry_2020_f1_LEC.parquet")
df.head(10)

Time,Speed,RPM,nGear,Throttle,Brake,X,Y,Z,Time_Sec
duration[ns],f64,f64,i64,f64,bool,f64,f64,f64,f64
0ns,302.704166,11918.206671,8,100.0,false,-975.26265,-376.235906,3028.122794,0.0
71ms,303.0,11913.0,8,100.0,false,-949.153334,-322.811421,3028.7839,0.071
207ms,303.566667,11920.48,8,100.0,false,-898.0,-220.0,3030.0,0.207
311ms,304.0,11926.2,8,100.0,false,-857.872403,-140.957297,3030.885684,0.311
427ms,304.0,11976.176667,8,100.0,false,-813.0,-53.0,3032.0,0.427
551ms,304.0,12029.6,8,100.0,false,-766.075722,39.97442,3033.559652,0.551
647ms,304.4,12025.64,8,100.0,false,-730.0,112.0,3035.0,0.647
791ms,305.0,12019.7,8,100.0,false,-675.667539,221.127492,3037.504216,0.791
867ms,305.633333,12041.296667,8,100.0,false,-647.0,279.0,3039.0,0.867


In [9]:
df = df.with_columns(
    (pl.col("nGear") - 2).clip(1, 6).alias("nGear")
)

In [10]:
df["nGear"].value_counts().sort("nGear")

nGear,count
i64,u32
2,55
3,126
4,115
5,253
6,114


decisamente meglio

per non ogni volta fare copia incolla del codice che visualizza il circuito e anche in ottica futura per usarlo in streamlit, creare la funzione che metterò in `src/utils`.

In [11]:
def plot_track_map(df, color_by="Speed", title="Track map"):
    fig = px.scatter(
        df,
        x="X",
        y="Y",
        color=color_by, # colora in base alla velocita
        title=title,
        color_continuous_scale="Turbo",  # rosso: veloce, blu:lento)
        width=800,
        height=800,
        hover_data=["RPM", "nGear", "Time_Sec"] # sono dati extraa qundo si passa col mouse sopra
    )
    
    fig.update_traces(marker=dict(size=3))

    fig.update_xaxes(visible=False) # toglie assi dal plot
    fig.update_yaxes(visible=False)

    fig.update_layout(
        template="plotly_dark",           
        plot_bgcolor="rgba(0,0,0,0)",     
        paper_bgcolor="rgba(0,0,0,0)",
        yaxis=dict(
            scaleanchor="x", 
            scaleratio=1  # scaleratio=1 così forza 1 metro su X a essere uguale a 1 metro su Y.
        )
    )

    return fig

In [12]:
from src.utils.plot_track_map import plot_track_map as plt_trckmp

fig = plt_trckmp(df, color_by="Speed", title="Mugello")
fig.show()